In [36]:
import librosa
import numpy as np
import tensorflow as tf
import os

In [37]:
SAMPLE_RATE = 8000
DURATION = 1.5
N_MFCC = 40

In [38]:
MODEL_PATH = "cortana_detector.keras"

In [39]:
AUDIO_FOLDER = "../samples/positive"

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH)

In [40]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)
    mfcc_mean = np.mean(mfcc.T, axis=0)  # flatten to 1D
    return mfcc_mean

In [41]:
for file in os.listdir(AUDIO_FOLDER):
    if not file.endswith(".wav"):
        continue

    path = os.path.join(AUDIO_FOLDER, file)
    features = extract_features(path)
    features = features.reshape(1, -1)  # reshape for model input

    prediction = model.predict(features, verbose=0)[0][0]
    label = "Cortana" if prediction > 0.5 else "Not Cortana"

    print(f"{file} → Prediction: {prediction:.3f} → Classified as: {label}")

90_sample.wav → Prediction: 0.998 → Classified as: Cortana
6_sample.wav → Prediction: 0.743 → Classified as: Cortana
399_sample.wav → Prediction: 1.000 → Classified as: Cortana
79_sample.wav → Prediction: 1.000 → Classified as: Cortana
349_sample.wav → Prediction: 0.204 → Classified as: Not Cortana
74_sample.wav → Prediction: 0.998 → Classified as: Cortana
324_sample.wav → Prediction: 0.759 → Classified as: Cortana
125_sample.wav → Prediction: 0.992 → Classified as: Cortana
332_sample.wav → Prediction: 0.983 → Classified as: Cortana
118_sample.wav → Prediction: 0.997 → Classified as: Cortana
232_sample.wav → Prediction: 0.989 → Classified as: Cortana
242_sample.wav → Prediction: 0.710 → Classified as: Cortana
114_sample.wav → Prediction: 1.000 → Classified as: Cortana
237_sample.wav → Prediction: 0.997 → Classified as: Cortana
4_sample.wav → Prediction: 0.998 → Classified as: Cortana
266_sample.wav → Prediction: 1.000 → Classified as: Cortana
222_sample.wav → Prediction: 0.646 → Classi